# 3. Wiener's Attack Solution _(4 points)_

In [20]:
# Using the decimal module to be able to operate over large numbers
from decimal import Decimal, getcontext

In [21]:
# Constant definitions
N = 31572253668946223368340157100721987766746218463736824783505635131921053880398741320644875421329504217933416422336918416263408538952177986508994175591182768950207147336188432455124667514201867376909227788134941323555902495049233601287696456816385455907725032886562051454322468668214625612317109993078620529639774616518443677804391408891641903175536847686030211632523660782037213447088479131269565204282324663517421706048402910651099744974056786727812991192176997050739471403369421617160003772289048267809022469915924569385942569022178774225834165571053657200129401052764120020877313983409004932571630004656678491554033
e = 7084215194443599160339614321138928983214362269664400507990258691986234567347379532809700232256805715669899435125455589766395600413533503573822989102823878826540036317848245491834763460693299412011619210827311587500311657621439893147815582844814748650942186449787061150827753958225319378164169085724521950445516816392443392835946057362175733538556192411994505858592612662805711714449145899544166354660465794567570468563034819166109105798194936416549149864412218709017864526810122862155857826601574302659442260666643066114221374852104428425503135373985569499838251381802559047735816838596220653454921257900296590700089

In [22]:
# Since the values of the constants "N" and "e" are very large, we have to increase the number of decimals to have a better precision of the number.
getcontext().prec = 700

Since we have the value of "N" and "e" values, we need to get the continued fractions expansion of the inital fraction $\frac{e}{N}$. In order to get the coeficients of the continued fraction expansion, we must return the coeficients of the divisions  

In [23]:
# Function to get a list of coefficients of the continued fractions expansion
def continued_fraction_expansion(numerator, denominator):
    quotient = numerator // denominator
    reminder = numerator % denominator
    coeficients = [quotient]

    while reminder != 0: # Avoid divison by zero
        numerator, denominator = denominator, reminder
        quotient = numerator // denominator
        reminder = numerator % denominator
        coeficients.append(quotient)

    return coeficients

Then we have to get the convergents of the coeficients by applying the following formula: if we have the continued fraction $[a_0,a_1,a_2,a_3]$ then the formula to get the first four convergents are $\frac{a_0}{1}, \frac{a_1a_0 + 1}{a_1}, \frac{a_2(a_1a_0 + 1) + a_0}{a_2a_1 + 1}, \frac{a_3(a_2(a_1a_0 + 1) + a_0) + (a_1a_0 + 1)}{a_3(a_2a_1 + 1) + a_1}$. (This information have been taken from https://en.wikipedia.org/wiki/Continued_fraction)

In [24]:
# Function to get the numerators and denominators
# We can get the convergents of the continued fractions by using the next form
def convergents_continued_fractions(continued_fractions_quotients):
    numerators = []
    denominators = []

    # Formula to get the numerator and denominator for the first coeficient
    numerators.append(continued_fractions_quotients[0])
    denominators.append(1)

    # Formula to get the numerator and denominator for the second coeficient
    numerators.append(continued_fractions_quotients[1]*continued_fractions_quotients[0] + 1)
    denominators.append(continued_fractions_quotients[1])

    # We start the loop from the third coeficient since the first 2 were already evaluated
    for i in range(2, len(continued_fractions_quotients)):
        numerator_i = continued_fractions_quotients[i]*numerators[i-1] + numerators[i-2]
        denominator_i = continued_fractions_quotients[i]*denominators[i-1] + denominators[i-2]

        numerators.append(numerator_i)
        denominators.append(denominator_i)
        yield (numerator_i, denominator_i)

In order to obtain the correct value of "d", we have to execute multiple steps which I describe below:
- First we need to get the convergents of the continued fractions $\frac{e}{N}$.
- From the convergents values obtained in the previous, the denominators would be the possible values for "d".
- We would iterate over all possible value "d" in order to validate if it is the correct one. In order to do so we will have to get the values of "p" and "q" by using the general solution of a quadratic equation using the following formulas described in the book "Cryptography Made Simple" page 303, $p = \frac{S + \sqrt{S^2 - 4 \cdot N}}{2}, q = \frac{S - \sqrt{S^2 - 4 \cdot N}}{2}$, after we obtain the values of "p" and "q" we can now check if the $p \cdot q = N$, if the equality is satisfied, then we know that the $d_i$ is the correct value for "d" we are looking for.

In [25]:
# Function containing the logic of the winner's attack
def wiener_attack(e, n):
    continued_fractions_coefficients = continued_fraction_expansion(e, n)
    continued_fractions_convergents = convergents_continued_fractions(continued_fractions_coefficients)
    for k, d in continued_fractions_convergents:
        if(k == 0): # Avoid division by zero
            continue
        edg = e * d - 1
        phi = edg // k

        # Get the two roots by using the general solution of a quadratic equation
        s = n + 1 - phi
        p = (Decimal(s) + Decimal((s**2) - 4 * n).sqrt()) / Decimal(2)
        q = (s - Decimal((s**2) - 4 * n).sqrt()) / 2

        # Checking correctness
        # When N == p * q we have found the correct d
        if p * q == N:
            return d, phi, p, q

Results of the Wiener's Attack

In [26]:
d, phi, p, q = wiener_attack(e, N)

print("Value of d: ", d)
print("Value of phi: ", phi)
print("Value of p: ", p)
print("Value of q: ", q)
print("Value of p*q: ", p*q)

Value of d:  13329884791477742620553494809186622521235859099452445414231999041472392302402686736103710652843340293147100831097333279613340600210383048977876617464069129
Value of phi:  31572253668946223368340157100721987766746218463736824783505635131921053880398741320644875421329504217933416422336918416263408538952177986508994175591182768950207147336188432455124667514201867376909227788134941323555902495049233601287696456816385455907725032886562051454322468668214625612317109993078620529639419232926990395584905455872993254526660377231090289138975380364480601633585065989511660413634549214566006930303971052158828015025689681827372866771654940553077358719287341114612631683840543406657891152271012641148167057623864102832389400174977751204554704568332779531958411688472768581360113099961766388298240
Value of p:  17914802359759001508469820559351739792131034188558747521081941386601796166478312143863861128727286177409821530671461468637118106905722169851083912190999490473941352453232399182102850198